# Proyecto: Extrayendo información de alto valor del registro de ventas en un restaurante

Algunas de las funciones que usremos son:

`.groupby(level=['column_name']).apply( lambda x: f(x) )` <- aplica la función f(x) a los registros agrupados por la columna ['column_name']

`pd.cut( df['column_name'] , bins = n)` <- clasifica los valores de la columna df['column_name'] de acuerdo a `n` intervalos del mismo tamaño

`pd.cut( df['column_name'] , bins = [x0,x1,x2,...,xn])` <- clasifica los valores de la columna df['column_name'] de acuerdo a `n` intervalos definidos por el usuario

`pd.cut( df['column_name'] , bins = n).value_counts()` <- Realiza un conteo de valores que cooresponden a cada intervalo de la partición 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
pd.options.display.float_format = '{:,.3f}'.format

En los siguientes datos se muestra el registro de ventas de un restaurante. Los datos incluyen: montos de cuentas pagadas, propinas, sexo del cliente, si es fumador o no, día de la venta, hora de comida y numero de personas que realizaron el consumo.

In [3]:
# Extraemos una base de datos sobre propinas 'tips' en un restaurante
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.990,1.010,Female,No,Sun,Dinner,2
1,10.340,1.660,Male,No,Sun,Dinner,3
2,21.010,3.500,Male,No,Sun,Dinner,3
3,23.680,3.310,Male,No,Sun,Dinner,2
4,24.590,3.610,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3
240,27.180,2.000,Female,Yes,Sat,Dinner,2
241,22.670,2.000,Male,Yes,Sat,Dinner,2
242,17.820,1.750,Male,No,Sat,Dinner,2


In [4]:
# Agregamos una columna de unos para realizar conteos:
df['ones'] = 1
df

,total_bill,tip,sex,smoker,day,time,size,ones
0,16.990,1.010,Female,No,Sun,Dinner,2,1
1,10.340,1.660,Male,No,Sun,Dinner,3,1
2,21.010,3.500,Male,No,Sun,Dinner,3,1
3,23.680,3.310,Male,No,Sun,Dinner,2,1
4,24.590,3.610,Female,No,Sun,Dinner,4,1
...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,1
240,27.180,2.000,Female,Yes,Sat,Dinner,2,1
241,22.670,2.000,Male,Yes,Sat,Dinner,2,1
242,17.820,1.750,Male,No,Sat,Dinner,2,1


In [5]:
# Extraemos un conteo (sobre la columna ['ones']) agrupado de acuerdo a las columnas ['sex','smoker']
df_g = df.groupby(['sex','smoker'])[['ones']].count()
df_g

ones
sex    smoker      
Male   Yes       60
       No        97
Female Yes       33
       No        54

Podemos ver los resultados de la tabla anterior como porcentajes, dividiendo entre el numéro total de hombres y mujeres:

In [6]:
# extraemos el total de hombres y mujeres:
tot_male_fem = df_g.groupby(['sex']).sum()
tot_male_fem

,ones
sex,
Male,157
Female,87


In [7]:
# Si dividimos los resultadoe entre el total de cada sexo, obtenemos los resultados como porcentajes:
(df_g / tot_male_fem ) *100

ones
sex    smoker       
Male   Yes    38.217
       No     61.783
Female Yes    37.931
       No     62.069

Una forma más rápida de obtener el resultado anterior es definir y aplicar una función con respecto a un nivel dado:

In [8]:
# Aplicamos una función con respecto a un nivel dado,
# en este caso  mostramos los resultados como porcentajes con respecto al sexo
df_g.groupby(level=['sex']).apply( lambda x: x/x.sum()*100)

ones
sex    smoker       
Male   Yes    38.217
       No     61.783
Female Yes    37.931
       No     62.069

## Clasificación con respecto a valores en intervalos dados:

In [9]:
# Para cada registro se muestra el intervalo al que pertenece cada valor de df['total_bill']:
pd.cut( df['total_bill'] , bins = 3)

0       (3.022, 18.983]
1       (3.022, 18.983]
2      (18.983, 34.897]
3      (18.983, 34.897]
4      (18.983, 34.897]
             ...       
239    (18.983, 34.897]
240    (18.983, 34.897]
241    (18.983, 34.897]
242     (3.022, 18.983]
243     (3.022, 18.983]
Name: total_bill, Length: 244, dtype: category
Categories (3, interval[float64]): [(3.022, 18.983] < (18.983, 34.897] < (34.897, 50.81]]

In [10]:
pd.cut( df['total_bill'] , bins = [3,20,35,60])

0       (3, 20]
1       (3, 20]
2      (20, 35]
3      (20, 35]
4      (20, 35]
         ...   
239    (20, 35]
240    (20, 35]
241    (20, 35]
242     (3, 20]
243     (3, 20]
Name: total_bill, Length: 244, dtype: category
Categories (3, interval[int64]): [(3, 20] < (20, 35] < (35, 60]]

In [11]:
# Extraemos el conteo de valores en cada clase:
pd.cut( df['total_bill'] , bins = 3).value_counts()

(3.022, 18.983]     140
(18.983, 34.897]     88
(34.897, 50.81]      16
Name: total_bill, dtype: int64

Hay 140 propinas en el primer intervalo, 88 en el segundo y 16 en el tercer intervalo

In [17]:
# podemos especificar el tamaño de cada intervalo:
# Extraemos el conteo de valores en cada clase:
pd.cut( df['total_bill'] , bins = [3,20,35,60]).value_counts()


(3, 20]     147
(20, 35]     81
(35, 60]     16
Name: total_bill, dtype: int64

In [13]:
# Agregamos una columna que incluye los intervalos anteriores:
df['bin_total'] = pd.cut( df['total_bill'] , bins = [3,20,35,60])
df

,total_bill,tip,sex,smoker,day,time,size,ones,bin_total
0,16.990,1.010,Female,No,Sun,Dinner,2,1,"(3, 20]"
1,10.340,1.660,Male,No,Sun,Dinner,3,1,"(3, 20]"
2,21.010,3.500,Male,No,Sun,Dinner,3,1,"(20, 35]"
3,23.680,3.310,Male,No,Sun,Dinner,2,1,"(20, 35]"
4,24.590,3.610,Female,No,Sun,Dinner,4,1,"(20, 35]"
...,...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,1,"(20, 35]"
240,27.180,2.000,Female,Yes,Sat,Dinner,2,1,"(20, 35]"
241,22.670,2.000,Male,Yes,Sat,Dinner,2,1,"(20, 35]"
242,17.820,1.750,Male,No,Sat,Dinner,2,1,"(3, 20]"


In [14]:
# Obtenemos la distribución de 'Lunch' y 'Dinner' en cada intervalo:
df_bins = df.groupby(['time','bin_total'])[['ones']].count()
df_bins

ones
time   bin_total      
Lunch  (3, 20]      52
       (20, 35]     14
       (35, 60]      2
Dinner (3, 20]      95
       (20, 35]     67
       (35, 60]     14

In [15]:
# Aplicamos una función con respecto a un nivel dado,
# en este caso  mostramos los resultados como porcentajes con respecto a 'time'
df_bins.groupby(level=['time']).apply( lambda x: x/x.sum()*100)

ones
time   bin_total       
Lunch  (3, 20]   76.471
       (20, 35]  20.588
       (35, 60]   2.941
Dinner (3, 20]   53.977
       (20, 35]  38.068
       (35, 60]   7.955

Interpretación de la tabla anterior:

A la hora del 'Lunch':

el 76.47% corresponde a facturas de entre 3 y 20 dolares, el 20.58% corresponde a facturas de entre 20 y 35 dolares, el 2.94% corresponde a facturas de entre 35 y 60 dolares

Análogamente para 'Dinner'
